In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import control as ctrl

# W(s) = 10/((s+1)(s+5))

t_ureg = 0.05      # требуемое время регулирования, с
pere_reg = 5       # требуемое перерегулирование, %
uy = 5             # величина управляющего воздействия

# Расчёт коэффициента демпфирования
xi = -np.log(pere_reg/100.0) / np.sqrt(np.pi**2 + np.log(pere_reg/100.0)**2)
print(f"Коэффициент демпфирования: {xi:.4f}")

# Расчёт собственной частоты по формуле лабораторной работы №3:
omega_n = 4 / (xi * t_ureg)
print(f"Собственная частота: {omega_n:.2f} рад/с")

# Для W(s)=10/(s+1)(s+5)
A = np.array([[-1, 0],
              [ 1, -5]])
B = np.array([[10],
              [ 0]])
C = np.array([[0, 1]])
D = np.array([[0]])

sys = ctrl.ss(A, B, C, D)

# Проверка полной управляемости объекта с помощью матрицы управляемости:
ctrb_matrix = ctrl.ctrb(A, B)
rank_ctrb = np.linalg.matrix_rank(ctrb_matrix)
print(f"Ранг матрицы управляемости: {rank_ctrb}")

# Постановка задачи модального управления.
# Требуется разместить определяющую комплексно-сопряжённую пару полюсов замкнутой системы:

s1 = -xi * omega_n + 1j * omega_n * np.sqrt(1 - xi**2)
s2 = -xi * omega_n - 1j * omega_n * np.sqrt(1 - xi**2)
desired_poles = [s1, s2]
print(f"Желаемые полюсы: {s1:.2f} и {s2:.2f}")

# Синтез регулятора методом размещения полюсов:
K = ctrl.place(A, B, desired_poles)
print(f"Коэффициенты регулятора: K = {K}")

# Формирование замкнутой системы с обратной связью.
# Замкнутая система описывается матрицей:
#   A_closed = A - B*K
# Для учёта величины управляющего воздействия (uy) корректно задаём вход:
#   B_closed = B * uy

A_closed = A - B @ K  # @ обеспечивает матричное умножение
B_closed = B * uy
closed_loop = ctrl.ss(A_closed, B_closed, C, D)

# Симуляция переходного процесса
t = np.linspace(0, 0.1, 1000)
t, y = ctrl.step_response(closed_loop, T=t)

# Расчёт установившегося значения, перерегулирования и времени регулирования:
steady_state = y[-1]
max_value = np.max(y)
actual_overshoot = (max_value - steady_state) / abs(steady_state) * 100

# Расчет времени регулирования по 5%-критерию:
tol = 0.05 * abs(steady_state)
idx = np.where(np.abs(y - steady_state) > tol)[0]
if len(idx) > 0 and idx[-1] < len(t) - 1:
    settling_time = t[idx[-1] + 1]
else:
    settling_time = t[-1]

print("\nРезультаты:")
print(f"Установившееся значение: {steady_state:.4f}")
print(f"Перерегулирование: {actual_overshoot:.2f}%")
print(f"Время регулирования: {settling_time:.4f} с")

plt.figure(figsize=(12, 6))
plt.plot(t, y, 'b-', label='Переходная характеристика')
plt.axhline(steady_state, color='r', linestyle='--', label='Установившееся значение')
plt.axhline(1.05 * steady_state, linestyle=':', label='+5% граница')
plt.axhline(0.95 * steady_state, linestyle=':', label='-5% граница')
plt.xlabel('Время, с')
plt.ylabel('Выход системы')
plt.title('Переходная характеристика САУ')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# W(s) = 50/((s+1)(s+10))

t_ureg = 0.1      # новое время регулирования, с
pere_reg = 5      # перерегулирование, %
uy = 3            # новая величина управляющего воздействия

xi = -np.log(pere_reg/100.0) / np.sqrt(np.pi**2 + np.log(pere_reg/100.0)**2)
print(f"Коэффициент демпфирования: {xi:.4f}")

omega_n = 4 / (xi * t_ureg)
print(f"Собственная частота: {omega_n:.2f} рад/с")

A = np.array([[-1, 0],
              [ 1, -10]])
B = np.array([[50],
              [ 0]])
C = np.array([[0, 1]])
D = np.array([[0]])

sys = ctrl.ss(A, B, C, D)

ctrb = ctrl.ctrb(A, B)
print(f"Ранг матрицы управляемости: {np.linalg.matrix_rank(ctrb)}")

s1 = -xi * omega_n + 1j * omega_n * np.sqrt(1 - xi**2)
s2 = -xi * omega_n - 1j * omega_n * np.sqrt(1 - xi**2)
desired_poles = [s1, s2]
print(f"Желаемые полюсы: {s1:.2f} и {s2:.2f}")

K = ctrl.place(A, B, desired_poles)
print(f"Коэффициенты регулятора: K = {K}")

A_closed = A - B @ K
B_closed = B * uy
closed_loop = ctrl.ss(A_closed, B_closed, C, D)

t = np.linspace(0, 0.2, 1000)
t, y = ctrl.step_response(closed_loop, T=t)

steady_state = y[-1]
max_value = np.max(y)
actual_overshoot = (max_value - steady_state) / abs(steady_state) * 100

tol = 0.05 * abs(steady_state)
idx = np.where(np.abs(y - steady_state) > tol)[0]
if len(idx) > 0 and idx[-1] < len(t) - 1:
    settling_time = t[idx[-1] + 1]
else:
    settling_time = t[-1]

print("\nРезультаты:")
print(f"Установившееся значение: {steady_state:.4f}")
print(f"Перерегулирование: {actual_overshoot:.2f}%")
print(f"Время регулирования: {settling_time:.4f} с")

plt.figure(figsize=(12, 6))
plt.plot(t, y, 'b-', linewidth=2, label='Переходная характеристика')
plt.axhline(steady_state, color='r', linestyle='--', linewidth=2, label='Установившееся значение')
plt.axhline(1.05 * steady_state, linestyle=':', label='+5% граница')
plt.axhline(0.95 * steady_state, linestyle=':', label='-5% граница')
plt.xlabel('Время, с')
plt.ylabel('Выход системы')
plt.title('Переходная характеристика САУ для нового объекта')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# W(s) = 5/((s+1)(s+10))

t_ureg = 0.05      # новое время регулирования, с
pere_reg = 1       # перерегулирование, %
uy = 3             # новая величина управляющего воздействия

xi = -np.log(pere_reg / 100.0) / np.sqrt(np.pi**2 + np.log(pere_reg / 100.0)**2)
print(f"Коэффициент демпфирования: {xi:.4f}")

omega_n = 4 / (xi * t_ureg)
print(f"Собственная частота: {omega_n:.2f} рад/с")

A = np.array([[-1, 0],
              [ 1, -10]])
B = np.array([[5],
              [0]])
C = np.array([[0, 1]])
D = np.array([[0]])

sys = ctrl.ss(A, B, C, D)

ctrb_matrix = ctrl.ctrb(A, B)
rank_ctrb = np.linalg.matrix_rank(ctrb_matrix)
print(f"Ранг матрицы управляемости: {rank_ctrb}")

s1 = -xi * omega_n + 1j * omega_n * np.sqrt(1 - xi**2)
s2 = -xi * omega_n - 1j * omega_n * np.sqrt(1 - xi**2)
desired_poles = [s1, s2]
print(f"Желаемые полюсы: {s1:.2f} и {s2:.2f}")

K = ctrl.place(A, B, desired_poles)
print(f"Коэффициенты регулятора: K = {K}")

A_closed = A - B @ K
B_closed = B * uy
closed_loop = ctrl.ss(A_closed, B_closed, C, D)

t = np.linspace(0, 0.15, 1000)
t, y = ctrl.step_response(closed_loop, T=t)

steady_state = y[-1]
max_value = np.max(y)
actual_overshoot = (max_value - steady_state) / abs(steady_state) * 100

tol = 0.05 * abs(steady_state)
idx = np.where(np.abs(y - steady_state) > tol)[0]
if len(idx) > 0 and idx[-1] < len(t) - 1:
    settling_time = t[idx[-1] + 1]
else:
    settling_time = t[-1]

print("\nРезультаты:")
print(f"Установившееся значение: {steady_state:.4f}")
print(f"Перерегулирование: {actual_overshoot:.2f}%")
print(f"Время регулирования: {settling_time:.4f} с")

# Построение графика
plt.figure(figsize=(12, 6))
plt.plot(t, y, 'b-', linewidth=2, label='Переходная характеристика')
plt.axhline(steady_state, color='r', linestyle='--', linewidth=2, label='Установившееся значение')
plt.axhline(1.01 * steady_state, linestyle=':', label='+1% граница')
plt.axhline(0.99 * steady_state, linestyle=':', label='-1% граница')
plt.xlabel('Время (с)')
plt.ylabel('Выход системы')
plt.title('Переходная характеристика САУ для нового объекта')
plt.legend()
plt.grid(True)
plt.show()


1. Коэффициент демпфирования ξ
Характеризует степень затухания колебаний в системе:

Если ξ < 1: система имеет колебательный переходной процесс (недодемпфированная).

Если ξ = 1: критическое демпфирование (быстрый выход на установившееся значение без колебаний).

Если ξ > 1: система переуспокоена (медленный монотонный переход).

Значение 0.8261 означает, что система имеет слабые затухающие колебания, что объясняет небольшое перерегулирование (1%).


2. Собственная частота ωₙ
Это частота свободных колебаний системы без учёта демпфирования. Чем выше ωₙ, тем быстрее система реагирует на входное воздействие.
В данном случае высокая частота (≈15.4 Гц) обеспечивает быстрое время регулирования 0.0366 с.


3. Ранг матрицы управляемости
Показывает, что система полностью управляема.

Ранг равен размерности пространства состояний (2), что позволяет произвольно размещать полюса замкнутой системы.

Если бы ранг был меньше 2, система была бы частично неуправляемой.


4. Желаемые полюсы
Корни характеристического уравнения замкнутой системы, определяющие её динамику:

Действительная часть (-80) отвечает за скорость затухания колебаний.

Мнимая часть (54.58) определяет частоту колебаний.

Расположение полюсов в левой полуплоскости гарантирует устойчивость системы.

5. Коэффициенты регулятора K
Это усиления обратной связи по состоянию, рассчитанные методом размещения полюсов:

K₁ = 29.8 – коэффициент для первого состояния (например, положения).

K₂ = 1575.69 – коэффициент для второго состояния (например, скорости).

Они обеспечивают перевод полюсов системы в желаемые позиции (-80 ± 54.58j).

6. Показатели качества системы
Установившееся значение (0.0016): выход системы в steady-state. Малое значение может быть связано с компенсацией управляющего воздействия (uy=3) или интегральной составляющей регулятора.